# Simplexn

In [ ]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

## simplexFacets(simplices::Cells)::Cells
Compute the `(d-1)`-skeleton (unoriented set of `facets`) of a simplicial `d`-complex.

In [ ]:
function simplexFacets(simplices)
	out = Array{Int64,1}[]
	for simplex in simplices
		for v in simplex
			facet = setdiff(simplex,v)
			push!(out, facet)
		end
	end
	# remove duplicate facets
	return sort(collect(Set(out)))
end

## simplex(n::Int, fullmodel=false::Bool)::Union{Lar.LAR, Lar.LARmodel}
Return a `LAR` model of the *`n`-dimensional simplex* in *`n`-space*.

When `fullmodel==true` return a `LARmodel`, including the faces, from dimension `1` to `n`.

In [ ]:
function simplex(n, fullmodel=false)
	eye(n) = LinearAlgebra.Matrix{Int}(I,n,n)
	V = [zeros(n,1) eye(n)]
	CV = [collect(1:n+1)]
	if fullmodel == false
		return V,CV
	else
		h = n
		cells = [CV]
		while h != 0
			push!(cells, simplexFacets(cells[end]))
			h -= 1
		end
		return V,reverse(cells)
	end
end

In [ ]:
@btime simplex(3, true)

## extrudeSimplicial(model::LAR, pattern::Array)::LAR
Algorithm for multimensional extrusion of a simplicial complex.
Can be applied to 0-, 1-, 2-, ... simplicial models, to get a 1-, 2-, 3-, .... model.

In [ ]:
function extrudeSimplicial(model::Lar.LAR, pattern)
    V = [model[1][:,k] for k=1:size(model[1],2)]
    FV = model[2]
    d, m = length(FV[1]), length(pattern)
    coords = collect(cumsum(append!([0], abs.(pattern))))
    offset, outcells, rangelimit, i = length(V), [], d*m, 0
    for cell in FV
        i += 1
        tube = [v+k*offset for k in range(0, length=m+1) for v in cell]
        cellTube = [tube[k:k+d] for k in range(1, length=rangelimit)]
        if i==1 outcells = reshape(cellTube, d, m)
        else outcells = vcat(outcells, reshape(cellTube, d, m)) end
    end
    cellGroups = []
    for i in 1:size(outcells, 2)
        if pattern[i]>0
            cellGroups = vcat(cellGroups, outcells[:, i])
        end
    end
    outVertices = [vcat(v, [z]) for z in coords for v in V]
    cellGroups = convert(Array{Array{Int, 1}, 1}, cellGroups)
    outModel = outVertices, cellGroups
    hcat(outVertices...), cellGroups
end
function extrudeSimplicial(model::Union{Any,Lar.Cells}, pattern)
    V,FV = model
    d, m = length(FV[1]), length(pattern)
    coords = collect(cumsum(append!([0], abs.(pattern))))
    offset, outcells, rangelimit, i = length(V), [], d*m, 0
    for cell in FV
        i += 1
        tube = [v+k*offset for k in range(0, length=m+1) for v in cell]
        cellTube = [tube[k:k+d] for k in range(1, length=rangelimit)]
        if i==1 outcells = reshape(cellTube, d, m)
        else outcells = vcat(outcells, reshape(cellTube, d, m)) end
    end
    cellGroups = []
    for i in 1:size(outcells, 2)
        if pattern[i]>0
            cellGroups = vcat(cellGroups, outcells[:, i])
        end
    end
    outVertices = [vcat(v, [z]) for z in coords for v in V]
    cellGroups = convert(Array{Array{Int, 1}, 1}, cellGroups)
    outModel = outVertices, cellGroups
    hcat(outVertices...), cellGroups
end

## simplexGrid(shape::Array)::LAR
Generate a simplicial complex decomposition of a cubical grid of ``d``-cuboids, where ``d`` is the length of `shape` array.

In [ ]:
function simplexGrid(shape)
    model = [[]], [[1]]
    for item in shape
        model = extrudeSimplicial(model, fill(1, item))
    end
    V, CV = model
    V = convert(Array{Float64,2}, V)
    return V, CV
end

In [ ]:
@btime simplexGrid([1,1,1])

## quads2triangles(quads::Cells)::Cells
Convert an array of *quads* with type `::Lar.Cells` into an array of *triangles* with the same type.

In [ ]:
function quads2triangles(quads::Lar.Cells)::Lar.Cells
	pairs = [[ Int[v1,v2,v3], Int[v3,v2,v4]] for (v1,v2,v3,v4) in quads ]
	return cat(pairs)
end